In [2]:
# Импорт необходимых библиотекрш
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from scipy import stats

# Загрузка тренировочных и тестовых данных
train_df = pd.read_csv('/kaggle/input/mai-ml-lab-1-biro/train.csv')
test_df = pd.read_csv('/kaggle/input/mai-ml-lab-1-biro/test.csv')

# Функция для предобработки данных
def preprocess_data(df, remove_outliers=True):
    df = df.copy()
    
    # Преобразование столбцов в нужные типы
    df['date_of_registration'] = pd.to_datetime(df['date_of_registration'], errors='coerce')
    df['complaints_count'] = pd.to_numeric(df['complaints_count'], errors='coerce')
    
    # Заполнение пропущенных значений медианами
    df['clicks'] = df['clicks'].fillna(df['clicks'].median())
    df['likes'] = df['likes'].fillna(df['likes'].median())
    df['complaints_count'] = df['complaints_count'].fillna(df['complaints_count'].median())
    
    # Обработка отрицательных значений в average_dwelltime
    median_dwelltime = df[df['average_dwelltime'] >= 0]['average_dwelltime'].median()
    df.loc[df['average_dwelltime'] < 0, 'average_dwelltime'] = median_dwelltime
    df['average_dwelltime'] = df['average_dwelltime'].fillna(median_dwelltime)
    
    # Удаление выбросов только из тренировочного набора данных
    if remove_outliers:
        threshold = 3
        features_to_check = ['clicks', 'likes', '4xx_errors', '5xx_errors', 'complaints_count', 'average_dwelltime']
        numeric_features = df[features_to_check].select_dtypes(include=[np.number])
        df = df[(np.abs(stats.zscore(numeric_features)) < threshold).all(axis=1)]
    
    # Удаление ненужных столбцов
    columns_to_drop = ['Unnamed: 0', 'date_of_registration']
    existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]
    df = df.drop(columns=existing_columns_to_drop)
    
    return df

# Предобработка тренировочных данных с удалением выбросов
train_df = preprocess_data(train_df, remove_outliers=True)

# Предобработка тестовых данных без удаления выбросов
test_df = preprocess_data(test_df, remove_outliers=False)

# Разделение признаков и целевой переменной
X_train = train_df.drop(columns=['source_attractiveness'])
y_train = train_df['source_attractiveness']
X_test = test_df.copy()

# Определение категориальных и числовых признаков
categorical_features = ['category']
numerical_features = X_train.columns.difference(categorical_features)

# Создание конвейера предобработки
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features),
        ('num', 'passthrough', numerical_features)
    ]
)

# Создание и обучение модели
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=200, random_state=42))
])

model.fit(X_train, y_train)

# Оценка модели с помощью кросс-валидации
mse_scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=5)
mean_mse = -mse_scores.mean()
print(f'Кросс-валидационное среднеквадратичное отклонение (MSE): {mean_mse:.4f}')

# Предсказания на тестовом наборе данных
y_predict = model.predict(X_test)

# Проверка количества строк
print(f"Количество строк в тестовом наборе: {X_test.shape[0]}")
print(f"Количество предсказаний: {len(y_predict)}")

# Создание файла для отправки без отдельного столбца 'ID'
submission_df = pd.DataFrame({'source_attractiveness': y_predict}, index=X_test.index)

# Сохранение файла с правильным разделителем и меткой индекса 'ID'
submission_df.to_csv('submission.csv', index_label='ID')

# Проверка файла отправки
print(f"Количество строк в файле submission.csv: {submission_df.shape[0]}")

Кросс-валидационное среднеквадратичное отклонение (MSE): 0.0131
Количество строк в тестовом наборе: 2000
Количество предсказаний: 2000
Количество строк в файле submission.csv: 2000
